In [1]:
import numpy as np
import pandas as pd
from sys import float_info
from sklearn import metrics
import os

In [2]:
def get_partial_auc_roc(labels, predictions):
    min_spec = 0.9
    p_aucroc = metrics.roc_auc_score(labels, predictions, max_fpr=(1 - min_spec))
    return p_aucroc

In [3]:
def screening_sens_at_spec(labels, predictions, at_spec=0.95, eps=float_info.epsilon):
    y_true = labels
    y_pred = predictions

    fpr, tpr, threshes = metrics.roc_curve(y_true, y_pred, drop_intermediate=False)
    spec = 1 - fpr

    operating_points_with_good_spec = spec >= (at_spec - eps)
    max_tpr = tpr[operating_points_with_good_spec][-1]

    operating_point = np.argwhere(operating_points_with_good_spec).squeeze()[-1]
    operating_tpr = tpr[operating_point]

    assert max_tpr == operating_tpr or (
            np.isnan(max_tpr) and np.isnan(operating_tpr)), f'{max_tpr} != {operating_tpr}'
    assert max_tpr == max(tpr[operating_points_with_good_spec]) or (
            np.isnan(max_tpr) and max(tpr[operating_points_with_good_spec])), \
        f'{max_tpr} == {max(tpr[operating_points_with_good_spec])}'

    return max_tpr

In [4]:
def _get_predition_joined_df(exp_dir, exp_id):
    prediction_files = [fn for fn in os.listdir(exp_dir) if fn.startswith('prediction') and exp_id in fn]
    dfs = []
    for fn in prediction_files:
        df = pd.read_csv(os.path.join(exp_dir, fn), index_col=0).set_index('filename')
        dfs.append(df)

    df_joined = dfs[0].labels
    for i, df in enumerate(dfs):
        df_joined = pd.merge(df_joined, df['predictions'].rename(f'pred_{i}'), left_index=True, right_index=True)

    df_joined['ensamble_prediction'] = df_joined.iloc[:, 1:].mean(axis=1)
    return df_joined

In [5]:
def get_metrics(exp_desc, exp_dir, exp_id):
    df_joined = _get_predition_joined_df(exp_dir, exp_id)
    pauroc = get_partial_auc_roc(df_joined.labels, df_joined.ensamble_prediction)
    sens_at_95_spec = screening_sens_at_spec(df_joined.labels, df_joined.ensamble_prediction)
    print(f'{exp_desc} ... pauroc {pauroc} ... sens_at_95_spec {sens_at_95_spec}')

In [6]:
print('=== exp rutger ===')
get_metrics('yolo baseline OD 1.2', 'exp_rutger', 'h0b29272b94d2f51a')
get_metrics('yolo tuned OD 1.2', 'exp_rutger', 'h0b29272b94d2f51a')
print()

print('=== exp equalisation ===')
get_metrics('IgnoreBlack OD 1.2', 'exp_equalisation', 'h699162adab7f35c3')
get_metrics('Yes OD 1.2', 'exp_equalisation', 'h76044e9d122a305e')
print()

print('=== exp crops ===')
get_metrics('OD 2.0', 'exp_crops', 'h2a932ca21a6567b5')
get_metrics('OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('OD 1.2', 'exp_crops', 'h5d2652f50e1bd942')
get_metrics('OD 1.0', 'exp_crops', 'hec9dc788fd725a32')
get_metrics('FOV 1.2', 'exp_crops', 'h5e77f73fb6cb1d37')
get_metrics('no crop', 'exp_crops', 'h76f5996b2c96146a')
print()

print('=== exp label smoothing ===')
get_metrics('base,   OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('LS 0.1, OD 1.5', 'exp_label_smoothing', 'h07af68425afb3d25')
get_metrics('LS 0.5, OD 1.5', 'exp_label_smoothing', 'hbe0385eca404c827')
print()

print('=== exp class imbalance ===')
get_metrics('base,                   OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('FL alpha=0.50, gamma=2, OD 1.5', 'exp_class_imbalance', 'h7610d694a6a38511')
get_metrics('FL alpha=0.66, gamma=2, OD 1.5', 'exp_class_imbalance', 'h08194bb70b6cd18c')
print()

print('=== exp aug rotations ===')
get_metrics('base,   OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('rot 10, OD 1.5', 'exp_rotations', 'h80e50dc8930f5af6')
get_metrics('rot 20, OD 1.5', 'exp_rotations', 'h1f9e2f2bd7c8736d')
get_metrics('rot 40, OD 1.5', 'exp_rotations', 'h458a6e2eae2eb24b')
print()

print('=== exp aug translations ===')
get_metrics('base,      OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('trans 0.1, OD 1.5', 'exp_translations', 'h71d58c99ab50c2e6')
get_metrics('trans 0.2, OD 1.5', 'exp_translations', 'hee6293b663deaf96')
get_metrics('trans 0.4, OD 1.5', 'exp_translations', 'h275dd8fff7dcdfcb')
print()

print('=== exp aug scale ===')
get_metrics('base,      OD 1.5', 'exp_crops', 'h8e023b47e3641bb7')
get_metrics('scale 0.1, OD 1.5', 'exp_aug_scale', 'h2faf4dc885212e0a')
get_metrics('scale 0.2, OD 1.5', 'exp_aug_scale', 'h20a4622db1030415')
get_metrics('scale 0.3, OD 1.5', 'exp_aug_scale', 'h9d3fe6e7b588c20f')
print()

print('=== exp crop finetuning ===')
get_metrics('OD 1.4', 'exp_crop_finetuning', 'ha21122c64b5683dc')
get_metrics('OD 1.5', 'exp_crop_finetuning', 'h6d2ee23d32630cd4')
get_metrics('OD 1.6', 'exp_crop_finetuning', 'ha133aee1531aa717')
print()


=== exp rutger ===
yolo baseline OD 1.2 ... pauroc 0.94279176201373 ... sens_at_95_spec 0.919732441471572
yolo tuned OD 1.2 ... pauroc 0.94279176201373 ... sens_at_95_spec 0.919732441471572

=== exp equalisation ===
IgnoreBlack OD 1.2 ... pauroc 0.9143601033225481 ... sens_at_95_spec 0.886435331230284
Yes OD 1.2 ... pauroc 0.9068447907183537 ... sens_at_95_spec 0.8643533123028391

=== exp crops ===
OD 2.0 ... pauroc 0.932744761783781 ... sens_at_95_spec 0.9053627760252366
OD 1.5 ... pauroc 0.9409274143375079 ... sens_at_95_spec 0.9274447949526814
OD 1.2 ... pauroc 0.9409732243154907 ... sens_at_95_spec 0.9053627760252366
OD 1.0 ... pauroc 0.940666173652255 ... sens_at_95_spec 0.917981072555205
FOV 1.2 ... pauroc 0.938876913388477 ... sens_at_95_spec 0.9212598425196851
no crop ... pauroc 0.8906552968777529 ... sens_at_95_spec 0.8391167192429022

=== exp label smoothing ===
base,   OD 1.5 ... pauroc 0.9409274143375079 ... sens_at_95_spec 0.9274447949526814
LS 0.1, OD 1.5 ... pauroc 0.938